In [10]:
import numpy as np
import os
import _pickle as cPickle
from collections import defaultdict
import torch
import torch.nn.init
from torch.autograd import Variable
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
dirname = './data_preprocessed_python'
filenames = os.listdir(dirname)
data_dic = defaultdict(lambda:[])
for i, filename in enumerate(filenames):
    full_filename = os.path.join(dirname, filename)
    x = cPickle.load(open(full_filename, 'rb'), encoding='ISO-8859-1')
    data_dic[filename[:-4]] = x

In [3]:
from scipy.stats import kurtosis, skew
def preprocessing(data):
    result = []
    mean = np.mean(data)
    median = np.median(data)
    maximum = np.amax(data)
    minimum = np.amin(data)
    std_dev = np.std(data)
    var = np.var(data)
    ran = np.ptp(data)
    skewness = skew(data)
    kurto = kurtosis(data)
    result=[mean, median, maximum, minimum,
                   std_dev, var, ran, skewness, kurto]
    return result

In [5]:
new_data_dic = defaultdict(lambda:defaultdict(lambda:[]))
for key in data_dic.keys():
    par = int(key[1:])
    for i, vid in enumerate(data_dic[key]['data']):
        new_data_dic[key]['data'].append([])
        for chnn in vid[:-8]:
            l =preprocessing(np.array(chnn))
            for batch in range(0,10):
                l = l+preprocessing(np.array(chnn[batch*807:max((batch+1)*807, 8064)]))
            new_data_dic[key]['data'][i].append(np.array([par, i+1]+l))
    for i,  vid in enumerate(data_dic[key]['labels']):
        l2 = []
        l3 = []
        new_data_dic[key]['labels'].append([])
        new_data_dic[key]['labels2'].append([])
        for value in vid:
            if value>=5:
                l2.append(1)
                l2.append(0)
            else:
                l2.append(0)
                l2.append(1)
            if value>=6:
                l3.append(1)
                l3.append(0)
                l3.append(0)
            elif value>=4 and value<6:
                l3.append(0)
                l3.append(1)
                l3.append(0)
            else:
                l3.append(0)
                l3.append(0)
                l3.append(1)
        new_data_dic[key]['labels'][i] += l2
        new_data_dic[key]['labels2'][i] += l3

In [6]:
print(len(new_data_dic['s01']['data'][0]))

32


In [180]:
data_loader = []
test_loader = []
idx = 0
print(len(new_data_dic.keys()))
for par in new_data_dic.keys():
    if par != 's02':
        for i, vid in enumerate(new_data_dic[par]['data']):
            data_loader.append((np.array(vid), new_data_dic[par]['labels'][i]))
            idx +=1
    else:
        for i, vid in enumerate(new_data_dic[par]['data']):
            test_loader.append((np.array(vid), new_data_dic[par]['labels'][i]))
            idx +=1
print(len(data_loader))
print(data_loader[1])

32
1240
(array([[  1.00000000e+00,   2.00000000e+00,  -8.27121269e-02, ...,
          3.70042683e+01,  -5.08154218e-02,   8.69235014e-01],
       [  1.00000000e+00,   2.00000000e+00,  -3.13505384e-02, ...,
          4.18200988e+01,   1.11964916e-01,   1.39442973e+00],
       [  1.00000000e+00,   2.00000000e+00,  -2.04664705e-02, ...,
          3.87172925e+01,   5.88444745e-02,   9.21659624e-01],
       ..., 
       [  1.00000000e+00,   2.00000000e+00,   6.82457494e-02, ...,
          2.08783110e+01,   1.04035679e-01,   1.35067118e-03],
       [  1.00000000e+00,   2.00000000e+00,   1.42352938e-01, ...,
          5.25603937e+01,   1.71642208e-01,   2.84520300e+00],
       [  1.00000000e+00,   2.00000000e+00,   1.56159050e-01, ...,
          3.58672933e+01,   3.79596399e-01,   9.76703888e-01]]), [1, 0, 1, 0, 1, 0, 1, 0])


In [181]:
data_loader2 = []
test_loader2 = []
idx = 0
print(len(new_data_dic.keys()))
for par in new_data_dic.keys():
    if par != 's02':
        for i, vid in enumerate(new_data_dic[par]['data']):
            data_loader2.append((np.array(vid), new_data_dic[par]['labels2'][i]))
            idx +=1
    else:
        for i, vid in enumerate(new_data_dic[par]['data']):
            test_loader2.append((np.array(vid), new_data_dic[par]['labels2'][i]))
            idx +=1
print(len(data_loader2))
print(data_loader2[1])

32
1240
(array([[  1.00000000e+00,   2.00000000e+00,  -8.27121269e-02, ...,
          3.70042683e+01,  -5.08154218e-02,   8.69235014e-01],
       [  1.00000000e+00,   2.00000000e+00,  -3.13505384e-02, ...,
          4.18200988e+01,   1.11964916e-01,   1.39442973e+00],
       [  1.00000000e+00,   2.00000000e+00,  -2.04664705e-02, ...,
          3.87172925e+01,   5.88444745e-02,   9.21659624e-01],
       ..., 
       [  1.00000000e+00,   2.00000000e+00,   6.82457494e-02, ...,
          2.08783110e+01,   1.04035679e-01,   1.35067118e-03],
       [  1.00000000e+00,   2.00000000e+00,   1.42352938e-01, ...,
          5.25603937e+01,   1.71642208e-01,   2.84520300e+00],
       [  1.00000000e+00,   2.00000000e+00,   1.56159050e-01, ...,
          3.58672933e+01,   3.79596399e-01,   9.76703888e-01]]), [1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0])


### Model

In [194]:
# this has all the main classes
import torch.nn as nn
# a neural network is a class derived from the core "Module" class in PyTorch
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=1,     
                out_channels=10,   
                kernel_size=3,     
                stride=2,          
                padding=1,         
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)), 
        )
        self.conv1_2d = nn.Sequential(  
            nn.Conv2d(
                in_channels=1,        
                out_channels=10,      
                kernel_size=3,        
                stride=2,             
                padding=2,            
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)),
        )
        
        self.conv2 = nn.Sequential(  
            nn.Conv1d(10, 20, 3, 2, 1),
            nn.ReLU(),                 
        )
        self.conv2_2d = nn.Sequential(  
            nn.Conv2d(10, 20, 3, 1, 1), 
            nn.ReLU(),                  
            nn.MaxPool2d(kernel_size=(2,1)),
        )
#         nn.ReLU()
        nn.Softmax()
        nn.Dropout(0.5)
#         self.out = nn.Softmax()
        self.out = nn.Linear(4160, 2)   # fully connected layer, output 1 class
#         self.out = nn.Softplus(4160,2)
    def forward(self, x):
        if (train2D):
            x = self.conv1_2d(x)
            x = self.conv2_2d(x)
        else:
            x = self.conv1(x)
            x = self.conv2(x)
#         x = x.view(x.size(0), -1)
        x = x.view(-1, 4160)# flatten the output 
        output = self.out(x)
        
        return output, x    # return x for potential visualization

# create an instance of the class
cnn = CNN()
# print the network architecture
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv1d(1, 10, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv1_2d): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(10, 20, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU()
  )
  (conv2_2d): Sequential(
    (0): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=4160, out_features=2, bias=True)
)


criterion sotfmarginLoss, MSELoss, ....

In [195]:
import torch.optim as optim

# standard cross entropy loss for classification
criterion = nn.BCEWithLogitsLoss()


# I tried this first with stochastic gradient descent, but it DOES NOT WORK!
optimizer = optim.SGD(cnn.parameters(), lr=0.0001, momentum=0.9)

# so I switched to a more advanced training scheme, which works like magic!
# optimizer = optim.Adam(cnn.parameters(),lr=0.00001, amsgrad=True)

# optimizer = torch.optim.RMSprop(cnn.parameters(), lr=learning_rate)


In [196]:
trX = []
trY = []
teX = []
teY = []
for i in data_loader:
    trX.append(i[0])
    trY.append(i[1][:2])
for i in test_loader:
    teX.append(i[0])
    teY.append(i[1][:2])
print(len(teY))

40


In [197]:
cnn.train()
train2D = True
# let's do 30 epochs
for epoch in range(5):
    print(epoch)
    # keep track of the current loss
    running_loss = 0.0
    training_error = 0.0
    # get the current minibatch
    for i, data in enumerate(data_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = torch.Tensor(np.array(inputs).reshape(-1, 1, 32, 101))
        labels = torch.Tensor(np.array(labels[:2]))
        # zero the parameter gradients
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            # forward + backward + optimize
            outputs = cnn(inputs)
            loss = criterion(outputs[0].view(-1), labels)
            loss.backward()
            optimizer.step()
        if i<=-1 and epoch<=-1:
            print(i,outputs[0],labels,loss.item())
        # training error
        pred_labels = np.sign(outputs[0].data.numpy().squeeze())
        
        training_error += np.sum(labels.numpy()!=pred_labels)/len(pred_labels)

        # print statistics
        running_loss += loss.item()
        
        if i % np.ceil(len(data_loader)/10) == np.ceil(len(data_loader)/10)-1: 

            running_loss = 0.0
            training_error = 0.0
print('Finished Training')

0
1
2
3
4
Finished Training


In [189]:
total_error=0
for inputs, labels in test_loader:
    inputs = torch.Tensor(np.array(inputs).reshape(-1, 1, 32, 101))
    labels = torch.Tensor(labels[:2])
    # push it through the network
    test_output, _ = cnn(inputs)
    # get the predicted labels by max activation
    #pred_labels = torch.max(test_output, 1)[1].data.numpy().squeeze()
    pred_labels = np.sign(test_output.data.numpy().squeeze())
    # get the error by comparing to the ground truth
    for i, label in enumerate(pred_labels):
        pred_labels[i] = max(0, label)
    total_error += 100*np.sum(labels.numpy()!=pred_labels)/len(pred_labels)
print('Error on testing set = {:.3f}%'.format(total_error/len(test_loader)))

Error on testing set = 37.500%


In [198]:
# this has all the main classes
import torch.nn as nn
# a neural network is a class derived from the core "Module" class in PyTorch
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=1,     
                out_channels=10,   
                kernel_size=3,     
                stride=2,          
                padding=1,         
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)), 
        )
        self.conv1_2d = nn.Sequential(  
            nn.Conv2d(
                in_channels=1,        
                out_channels=10,      
                kernel_size=3,        
                stride=2,             
                padding=2,            
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,1)),
        )
        
        self.conv2 = nn.Sequential(  
            nn.Conv1d(10, 20, 3, 2, 1),
            nn.ReLU(),                 
        )
        self.conv2_2d = nn.Sequential(  
            nn.Conv2d(10, 20, 3, 1, 1), 
            nn.ReLU(),                  
            nn.MaxPool2d(kernel_size=(2,1)),
        )
#         nn.ReLU()
        nn.Softmax()
        nn.Dropout(0.5)
#         self.out = nn.Softmax()
        self.out = nn.Linear(4160, 3)   # fully connected layer, output 1 class
#         self.out = nn.Softplus(4160,2)
    def forward(self, x):
        if (train2D):
            x = self.conv1_2d(x)
            x = self.conv2_2d(x)
        else:
            x = self.conv1(x)
            x = self.conv2(x)
#         x = x.view(x.size(0), -1)
        x = x.view(-1, 4160)# flatten the output 
        output = self.out(x)
        
        return output, x    # return x for potential visualization

# create an instance of the class
cnn = CNN()
# print the network architecture
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv1d(1, 10, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv1_2d): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(10, 20, kernel_size=(3,), stride=(2,), padding=(1,))
    (1): ReLU()
  )
  (conv2_2d): Sequential(
    (0): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=4160, out_features=3, bias=True)
)


In [202]:
cnn.train()

for epoch in range(5):
    print(epoch)
    # keep track of the current loss
    running_loss = 0.0
    training_error = 0.0
    # get the current minibatch
    for i, data in enumerate(data_loader2, 0):
        # get the inputs
        inputs, labels = data
        inputs = torch.Tensor(np.array(inputs).reshape(-1, 1, 32, 101))
        labels = torch.Tensor(np.array(labels[:3]))
        # zero the parameter gradients
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            # forward + backward + optimize
            outputs = cnn(inputs)
            
            loss = criterion(outputs[0].view(-1), labels)
            loss.backward()
            optimizer.step()
            
        if i<=-1 and epoch<=-1:
            print(i,outputs[0],labels,loss.item())
        # training error
        pred_labels = np.sign(outputs[0].data.numpy().squeeze())
        training_error += np.sum(labels.numpy()!=pred_labels)/len(pred_labels)
        # print statistics
        running_loss += loss.item()
        
        if i % np.ceil(len(data_loader2)/10) == np.ceil(len(data_loader2)/10)-1: 

            running_loss = 0.0
            training_error = 0.0

print('Finished Training')

0
1
2
3
4
Finished Training


3 class - test

In [204]:
total_error=0
for inputs, labels in test_loader2:
    inputs = torch.Tensor(np.array(inputs).reshape(-1, 1, 32, 101))
    labels = torch.Tensor(labels)
    # push it through the network
    test_output, _ = cnn(inputs)
    # get the predicted labels by max activation
    #pred_labels = torch.max(test_output, 1)[1].data.numpy().squeeze()
    pred_labels = np.sign(test_output.data.numpy().squeeze())
    for i, label in enumerate(pred_labels):
        pred_labels[i] = max(0, label)
    # get the error by comparing to the ground truth
    total_error += 100*np.sum(labels.numpy()!=pred_labels)/len(pred_labels)
print('Error on testing set = {:.3f}%'.format(total_error/len(test_loader)))

Error on testing set = 33.333%


D:\download\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  del sys.path[0]
